# Speech Denoising using RNN

In [1]:
import os
from os import listdir
from os.path import isfile, join
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import tensorflow as tf
import numpy as np
import time
import math
import pickle
import librosa
import matplotlib.pyplot as plt
%matplotlib notebook

## Reading Training Data
Reading the processed training data from pickle files, that are saved earlier (data_prep_part2.ipynb).

In [2]:
s_spect_train = pickle.load(open('train_source', 'rb'))
n_spect_train = pickle.load(open('train_noise', 'rb'))
x_spect_train = pickle.load(open('train_mix', 'rb'))
s_train_vec = pickle.load(open('istft_source', 'rb'))
x_phase_train = pickle.load(open('train_mix_phase', 'rb'))

## Reading Validation Data
Reading the processed validation data from pickle files, that are saved earlier (data_prep_part2.ipynb).

In [3]:
s_spect_valid = pickle.load(open('valid_source', 'rb'))
n_spect_valid = pickle.load(open('valid_noise', 'rb'))
x_spect_valid = pickle.load(open('valid_mix', 'rb'))
s_valid_vec = pickle.load(open('istft_valid', 'rb'))
x_phase_valid = pickle.load(open('valid_mix_phase', 'rb'))

## IBM matrix
- Number of timesteps are fixed to 150, so any signal that is less than this values is padded and any signal that is greater than this value is truncated.
- IBM matrix is calculated on this modified data.

In [4]:
time_steps = 150

In [5]:
y_spect_train = [] #list of IBM matrices for signals in training data
for i in range(len(s_spect_train)):
    pad_len = time_steps - s_spect_train[i].shape[1]
    if(pad_len <= 0):
        s_spect_train[i] = s_spect_train[i][:,0:time_steps]
        n_spect_train[i] = n_spect_train[i][:,0:time_steps]
        x_spect_train[i] = x_spect_train[i][:,0:time_steps]
        x_phase_train[i] = x_phase_train[i][:,0:time_steps]
    else:
        s_spect_train[i] = np.pad(s_spect_train[i], ((0,0),(0,pad_len)), 'constant')
        n_spect_train[i] = np.pad(n_spect_train[i], ((0,0),(0,pad_len)), 'constant')
        x_spect_train[i] = np.pad(x_spect_train[i], ((0,0),(0,pad_len)), 'constant')
        x_phase_train[i] = np.pad(x_phase_train[i], ((0,0),(0,pad_len)), 'constant')
    y_spect_train.append((s_spect_train[i] > n_spect_train[i]).astype(int))

In [6]:
y_spect_valid = [] #list of IBM matrices for signals in validation data
for i in range(len(s_spect_valid)):
    pad_len = time_steps - s_spect_valid[i].shape[1]
    if(pad_len <= 0):
        s_spect_valid[i] = s_spect_valid[i][:,0:time_steps]
        n_spect_valid[i] = n_spect_valid[i][:,0:time_steps]
        x_spect_valid[i] = x_spect_valid[i][:,0:time_steps]
        x_phase_valid[i] = x_phase_valid[i][:,0:time_steps]
    else:
        s_spect_valid[i] = np.pad(s_spect_valid[i], ((0,0),(0,pad_len)), 'constant')
        n_spect_valid[i] = np.pad(n_spect_valid[i], ((0,0),(0,pad_len)), 'constant')
        x_spect_valid[i] = np.pad(x_spect_valid[i], ((0,0),(0,pad_len)), 'constant')
        x_phase_valid[i] = np.pad(x_phase_valid[i], ((0,0),(0,pad_len)), 'constant')
    y_spect_valid.append((s_spect_valid[i] > n_spect_valid[i]).astype(int))

As we have a list of 2D numpy arrays, they are coverted to 3D arrays to make it compatible with the model. Now both input and output data will be in format \[batch_size, time_steps, features\] 

In [7]:
x_spect_train_arr = np.dstack(x_spect_train)
y_spect_train_arr = np.dstack(y_spect_train)
x_spect_train_arr=np.rollaxis(x_spect_train_arr,-1)
x_spect_train_arr = np.transpose(x_spect_train_arr, (0,2,1))
y_spect_train_arr=np.rollaxis(y_spect_train_arr,-1)
y_spect_train_arr = np.transpose(y_spect_train_arr, (0,2,1))
x_phase_train_arr = np.dstack(x_phase_train)
x_phase_train_arr = np.rollaxis(x_phase_train_arr,-1)
x_phase_train_arr = np.transpose(x_phase_train_arr, (0,2,1))

In [8]:
x_spect_valid_arr = np.dstack(x_spect_valid)
y_spect_valid_arr = np.dstack(y_spect_valid)
x_spect_valid_arr=np.rollaxis(x_spect_valid_arr,-1)
x_spect_valid_arr = np.transpose(x_spect_valid_arr, (0,2,1))
y_spect_valid_arr=np.rollaxis(y_spect_valid_arr,-1)
y_spect_valid_arr = np.transpose(y_spect_valid_arr, (0,2,1))
x_phase_valid_arr = np.dstack(x_phase_valid)
x_phase_valid_arr = np.rollaxis(x_phase_valid_arr,-1)
x_phase_valid_arr = np.transpose(x_phase_valid_arr, (0,2,1))

In [9]:
print(x_spect_train_arr.shape)
print(y_spect_train_arr.shape)
print(x_phase_train_arr.shape)

(1200, 150, 513)
(1200, 150, 513)
(1200, 150, 513)


In [10]:
print(x_spect_valid_arr.shape)
print(y_spect_valid_arr.shape)
print(x_phase_valid_arr.shape)

(1200, 150, 513)
(1200, 150, 513)
(1200, 150, 513)


## Model
- Placeholders: Both input and output data has similar dimensions as we are doing a sort of regression problem. As the batch_size could vary it is given as None in the placeholder.
- During execution a batch_size of 10 is used.
- 2 RNN layers are stacked in the network. The first cell hass 500 units and the second cell has 513 units so as to match out output feature dimension and avoid a fully connected layer.
- GRU cells are used in both rnn layers. The activation the the second layer is changed to sigmoid as out output is binary valued.
- Mean squared error is used in the cost function, as it is a sort of regression problem.
- Adagrad optimizer is used with a learning rate of $0.5$

In [11]:
batch_size = 10
input_dim = 513

In [12]:
# Batch size x time steps x features.
data = tf.placeholder(tf.float32, [None, time_steps, input_dim])
y = tf.placeholder(tf.float32, [None, time_steps, input_dim])

In [13]:
gru_sizes = [500,513]
GRUs = [tf.contrib.rnn.GRUCell(size, activation=tf.nn.sigmoid) if size==513 else tf.contrib.rnn.GRUCell(size) for size in gru_sizes]
#drops = [tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=0.8) for lstm in lstms]
cell = tf.contrib.rnn.MultiRNNCell(GRUs)
# initial_state = cell.zero_state(batch_size, tf.float32)
output, state = tf.nn.dynamic_rnn(cell, data, dtype=tf.float32)

In [14]:
cost = tf.losses.mean_squared_error(output, y)

In [15]:
optimizer = tf.train.AdagradOptimizer(0.5).minimize(cost)

In [16]:
#Setting configurtion for tensorflow memory usage
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.33

#Creating a tendorflow session
sess=tf.Session(config=config)

## Executing Model
Model is executed for 1000 epochs, and SNR values for validation signals are computed for every 100 epochs.

As I have used truncated signals, the predicted signal will have different size than the ground truth source signals depending on the length of original signal. So I have computed two different SNR values for the validation data, first one ("SNR validation") keeping the source signal as it is and padding or truncating the predicted signal to match the size of the source signal. In the second one ("SNR validation trunc"), I took the minimum size between both source and predicted signals and truncated the longer one.

The first approach is more reliable and it produces a SNR value of $10.5$ for the validation data.

In [17]:
sess.run(tf.global_variables_initializer())
num_epochs = 1000
num_batches = int(x_spect_train_arr.shape[0]/batch_size)
strt = time.clock()
for epoch in range(num_epochs):
    estrt = time.clock()
    epoch_cost = 0
    for i in range(num_batches):
        start = i*batch_size
        batch_ip = x_spect_train_arr[start:start+batch_size, :, :]
        batch_op = y_spect_train_arr[start:start+batch_size, :, :]
        sess.run(optimizer, feed_dict={data:batch_ip, y:batch_op})
    snr_p_v = 0
    snr_p_v_min = 0
    if (epoch+1)%10 == 0: print("Epochs Done: ", epoch)
    if (epoch+1)%100 == 0:
        y_p_v = sess.run(output, feed_dict={data:x_spect_valid_arr[:,:,:], y:y_spect_valid_arr[:,:,:]})
        xy_p_v = y_p_v * x_spect_valid_arr[:,:,:] * x_phase_valid_arr[:,:,:]
        for j in range(1200):
            xy_test_p_v = librosa.istft(xy_p_v[j,:,:].T, hop_length=512)
            numer_v = np.sum(np.square(s_valid_vec[j]))
            pad_p_v = len(s_valid_vec[j]) - len(xy_test_p_v)
            min_p_v = min(len(s_valid_vec[j]), len(xy_test_p_v))
            denom_p_v_min = np.sum(np.square(s_valid_vec[j][:min_p_v] - xy_test_p_v[:min_p_v]))
            if pad_p_v > 0: 
                xy_test_p_v = np.pad(xy_test_p_v, (0,pad_p_v), 'constant')
                denom_p_v = np.sum(np.square(s_valid_vec[j] - xy_test_p_v))
            else:
                denom_p_v = np.sum(np.square(s_valid_vec[j] - xy_test_p_v[0:len(s_valid_vec[j])]))
            snr_p_v += 10*np.log10(numer_v/denom_p_v)
            snr_p_v_min += 10*np.log10(numer_v/denom_p_v_min)
        print("Epoch:", epoch, ", SNR validation:", round(snr_p_v/1200,3), ", SNR validation trunc:", round(snr_p_v_min/1200,3))
        print("====================================================")

Epochs Done:  9
Epochs Done:  19
Epochs Done:  29
Epochs Done:  39
Epochs Done:  49
Epochs Done:  59
Epochs Done:  69
Epochs Done:  79
Epochs Done:  89
Epochs Done:  99
Epoch: 99 , SNR validation: 8.923 , SNR validation trunc: 9.411
Epochs Done:  109
Epochs Done:  119
Epochs Done:  129
Epochs Done:  139
Epochs Done:  149
Epochs Done:  159
Epochs Done:  169
Epochs Done:  179
Epochs Done:  189
Epochs Done:  199
Epoch: 199 , SNR validation: 9.761 , SNR validation trunc: 10.314
Epochs Done:  209
Epochs Done:  219
Epochs Done:  229
Epochs Done:  239
Epochs Done:  249
Epochs Done:  259
Epochs Done:  269
Epochs Done:  279
Epochs Done:  289
Epochs Done:  299
Epoch: 299 , SNR validation: 10.126 , SNR validation trunc: 10.709
Epochs Done:  309
Epochs Done:  319
Epochs Done:  329
Epochs Done:  339
Epochs Done:  349
Epochs Done:  359
Epochs Done:  369
Epochs Done:  379
Epochs Done:  389
Epochs Done:  399
Epoch: 399 , SNR validation: 10.275 , SNR validation trunc: 10.871
Epochs Done:  409
Epochs Do

## Testing
Test data is read from the earlier stored pickle files. The data is passed to above model to compute the prediction mask. It is then multiplied to the spectrogram and its phase information, later inverse STFT is applied to this and saved to audio files.

The file naming is in format **denoise\_tex$<$filenumber$>$.wav** files 

In [19]:
x_spect_test = pickle.load(open('test_mix', 'rb'))
x_phase_test = pickle.load(open('test_mix_phase', 'rb'))

In [22]:
for i in range(len(x_spect_test)):
    pad_len = time_steps - x_spect_test[i].shape[1]
    if(pad_len <= 0):
        x_spect_test[i] = x_spect_test[i][:,0:time_steps]
        x_phase_test[i] = x_phase_test[i][:,0:time_steps]
    else:
        x_spect_test[i] = np.pad(x_spect_test[i], ((0,0),(0,pad_len)), 'constant')
        x_phase_test[i] = np.pad(x_phase_test[i], ((0,0),(0,pad_len)), 'constant')

In [24]:
x_spect_test_arr = np.dstack(x_spect_test)
x_spect_test_arr = np.rollaxis(x_spect_test_arr,-1)
x_spect_test_arr = np.transpose(x_spect_test_arr, (0,2,1))
x_phase_test_arr = np.dstack(x_phase_test)
x_phase_test_arr = np.rollaxis(x_phase_test_arr,-1)
x_phase_test_arr = np.transpose(x_phase_test_arr, (0,2,1))

In [25]:
print(x_spect_test_arr.shape)
print(x_phase_test_arr.shape)

(400, 150, 513)
(400, 150, 513)


In [ ]:
test_path = '/opt/e533/timit-homework/te/'
x_files_test = []
x_files_test_denoise = []
for f in listdir(test_path):
    if isfile(join(test_path, f)):
        x_files_test.append(join(test_path,f))
        x_files_test_denoise.append(join('dtest','denoise_'+f))
x_files_test.sort()
x_files_test_denoise.sort()

In [45]:
y_p_te = sess.run(output, feed_dict={data:x_spect_test_arr[:,:,:]})
xy_p_te = y_p_te * x_spect_test_arr[:,:,:] * x_phase_test_arr[:,:,:]
for j in range(400):
    sn, sr=librosa.load(x_files_test[j])
    xy_test_p_te = librosa.istft(xy_p_te[j,:,:].T, hop_length=512)
    librosa.output.write_wav(x_files_test_denoise[j], xy_test_p_te, sr)

## References
1. https://www.tensorflow.org/tutorials/recurrent
2. https://danijar.com/introduction-to-recurrent-networks-in-tensorflow/
3. https://jasdeep06.github.io/posts/Understanding-LSTM-in-Tensorflow-MNIST/